In [95]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/MachineLearning/Module_3_Bank_Proj/Brandon

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Github/MachineLearning/Module_3_Bank_Proj/Brandon


In [96]:
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv")

In [97]:
#Profiling of Dataset

#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 
#from pandas_profiling import ProfileReport as pr
#profile = pr(data)
#profile.to_notebook_iframe()

In [98]:
### Geo Location Pre-Processing ###

In [99]:
import altair as alt

from altair import pipe, limit_rows, to_values
t = lambda data: pipe(data, limit_rows(max_rows=20000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')

data = data[data.lat > 45]
data = data[data.long < -110]

data_high = data[data.price > 500000]
data_low = data[data.price <= 500000]

high_chart = alt.Chart(data_high).mark_circle().encode(
    alt.X("long:Q",
          scale=alt.Scale(zero = False)),
    alt.Y("lat:Q",
          scale=alt.Scale(zero = False)),
          color = "price"
)

low_chart = alt.Chart(data_low).mark_circle().encode(
    alt.X("long:Q",
          scale=alt.Scale(zero = False)),
    alt.Y("lat:Q",
          scale=alt.Scale(zero = False)),
          color = "price"
)

high_chart


Output hidden; open in https://colab.research.google.com to view.

In [100]:
low_chart

Output hidden; open in https://colab.research.google.com to view.

In [101]:
alt.Chart(data).mark_rect().encode(
    alt.X('long:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('lat:Q', bin=alt.Bin(maxbins=100)),
    alt.Color('mean(price):Q', scale=alt.Scale(scheme='greenblue'))
)

Output hidden; open in https://colab.research.google.com to view.

In [102]:
alt.Chart(data).mark_rect().encode(
    alt.X('long:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('lat:Q', bin=alt.Bin(maxbins=100)),
    alt.Color('count(id):Q', scale=alt.Scale(scheme='greenblue'))
)

Output hidden; open in https://colab.research.google.com to view.

In [103]:
data['lat_bin'] = 0
data['long_bin'] = 0
for lat in range(4715,4779,1):
  if lat + 1 > 4779:
    break
  else:
    data.loc[(data.lat >= (lat/100)) & (data.lat < ((lat/100) + .01)),'lat_bin'] = lat/100
  
for long in range(-12252,-12124,2):
  if long + 1 > -12124:
    break
  else:
    data.loc[(data.long >= (long/100)) & (data.long < ((long/100) + .02)),'long_bin'] = long/100
  

In [104]:
(data.groupby(["lat_bin","long_bin"], as_index=False)
  .agg({"price":"mean", "id":"count"})
  .rename(columns = {"price":"avg_neighborhood_price", "id":"no_houses_neighborhood"}))


,lat_bin,long_bin,avg_neighborhood_price,no_houses_neighborhood
0,47.15,-121.96,687000.000000,1
1,47.15,-121.66,380000.000000,1
2,47.16,-121.98,542500.000000,2
3,47.17,-122.04,360000.000000,1
4,47.17,-122.02,230000.000000,1
...,...,...,...,...
1181,47.77,-122.06,849333.333333,3
1182,47.77,-122.04,603750.000000,2
1183,47.77,-121.94,637000.000000,1
1184,47.77,-121.92,309637.500000,2


In [106]:
data["avg_neighbor_price"] = data.groupby(["lat_bin", "long_bin"])["price"].transform("mean")

In [112]:
alt.Chart(data).mark_circle().encode(
    alt.X("avg_neighbor_price:Q", scale = alt.Scale(type='log')),
    alt.Y("price:Q", scale = alt.Scale(type='log'))
).configure_mark(
    opacity=0.1,
    color='red'
)

Output hidden; open in https://colab.research.google.com to view.